In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from pathlib import Path

In [29]:
# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

In [15]:
import pathlib 
import numpy as np

data_dir = pathlib.Path("/kaggle/input/100-bird-species/train")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")])) # creating a list of class names from subdirectory 
len(class_names)

525

In [2]:
# Define device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [3]:
# Define data paths
train_path = "/kaggle/input/100-bird-species/train"
val_path = "/kaggle/input/100-bird-species/valid"
test_path = "/kaggle/input/100-bird-species/test"

In [4]:
# Define batch size
batch_size = 64

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [5]:
# Load datasets
train_dataset = torchvision.datasets.ImageFolder(root=train_path, transform=transform)
val_dataset = torchvision.datasets.ImageFolder(root=val_path, transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=test_path, transform=transform)

In [6]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [12]:
# Download the pretrained weights for EfficientNet_B0
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # NEW in torchvision 0.13, "DEFAULT" means "best weights available"

In [13]:
# Load EfficientNet_B0 model with pretrained weights
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [8]:
# Freeze the pretrained parameters
for param in model.parameters():
    param.requires_grad = False

In [16]:
# Update the classifier head to suit our problem
model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=len(class_names),
              bias=True).to(device))

In [30]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=model,
        input_size=(64, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [64, 3, 224, 224]    [64, 525]            --                   True
├─Sequential (features)                                      [64, 3, 224, 224]    [64, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [64, 3, 224, 224]    [64, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [64, 3, 224, 224]    [64, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [64, 32, 112, 112]   [64, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [64, 32, 112, 112]   [64, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [64, 32, 112, 112]   [64, 16, 112

In [17]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    # Set model to training mode
    model.train()

    running_loss = 0.0
    correct_predictions = 0

    # Training phase
    for images, labels in tqdm(train_loader, desc=f'Training - Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct_predictions.double() / len(train_dataset)
    print(f'Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_correct_predictions = 0

    with torch.no_grad():
        for val_images, val_labels in tqdm(val_loader, desc=f'Validation - Epoch {epoch + 1}/{num_epochs}', unit='batch'):
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)

            val_running_loss += val_loss.item() * val_images.size(0)
            _, val_preds = torch.max(val_outputs, 1)
            val_correct_predictions += torch.sum(val_preds == val_labels.data)

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_accuracy = val_correct_predictions.double() / len(val_dataset)
    print(f'Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.4f}')

print('Training complete')


Epoch 1/5
----------


Training - Epoch 1/5: 100%|██████████| 1323/1323 [14:51<00:00,  1.48batch/s]


Training Loss: 1.0149, Accuracy: 0.7765


Validation - Epoch 1/5: 100%|██████████| 42/42 [00:21<00:00,  1.92batch/s]


Validation Loss: 0.2464, Accuracy: 0.9364
Epoch 2/5
----------


...

Validation Loss: 0.1977, Accuracy: 0.9444
Epoch 5/5
----------


Training - Epoch 5/5: 100%|██████████| 1323/1323 [07:53<00:00,  2.80batch/s]


Training Loss: 0.1437, Accuracy: 0.9575


Validation - Epoch 5/5: 100%|██████████| 42/42 [00:09<00:00,  4.65batch/s]

Validation Loss: 0.1519, Accuracy: 0.9623
Training complete


In [19]:
# Test phase
model.eval()
test_correct_predictions = 0

with torch.no_grad():
    for test_images, test_labels in tqdm(test_loader, desc='Testing', unit='batch'):
        test_images = test_images.to(device)
        test_labels = test_labels.to(device)

        test_outputs = model(test_images)
        _, test_preds = torch.max(test_outputs, 1)

        test_correct_predictions += torch.sum(test_preds == test_labels.data)

test_accuracy = test_correct_predictions.double() / len(test_dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')


Testing: 100%|██████████| 42/42 [00:25<00:00,  1.66batch/s]

Test Accuracy: 0.9710


In [35]:
import requests
from PIL import Image
from io import BytesIO


# Function to load image from URL or local file path
def load_image(image_path):
    if image_path.startswith('http'):
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(image_path)
    return image

# URL or local file path of the image you want to test
image_path = "https://th.bing.com/th/id/R.3678f60ff36f2673189180b788e7485b?rik=XKeHnRUjLcNI%2bg&pid=ImgRaw&r=0"  # Update with your image URL or file path

# Load the image
image = load_image(image_path)

# Preprocess the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
preprocessed_image = transform(image).unsqueeze(0)  # Add batch dimension

# Move preprocessed image to the appropriate device
preprocessed_image = preprocessed_image.to(device)

# Pass the preprocessed image through the model
with torch.no_grad():
    model.eval()  # Set model to evaluation mode
    outputs = model(preprocessed_image)

# Get the predicted class
_, predicted = torch.max(outputs, 1)
predicted_class = class_names[predicted.item()]

print("Predicted class:", predicted_class)


Predicted class: ASIAN GREEN BEE EATER


In [21]:
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                          exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
               f=model_save_path)


In [23]:
save_model(model,'/kaggle/working/','birdingEff.pth')

[INFO] Saving model to: /kaggle/working/birdingEff.pth


In [24]:
# Calculate model size
model_size = Path('/kaggle/working/birdingEff.pth').stat().st_size / (1024*1024)  # in MB
print(f"Model size: {model_size:.2f} MB")

Model size: 18.14 MB
